In [10]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib widget

# $H_20$ grouping vs $x$ (w/ shuffle)

In [2]:
file = 'H20_grouping_shuffle_ideal_vs_connectivity'
data = np.load(file + '.npy', allow_pickle=True).item()

print(data.keys())

for key in data:
    vars()[key] = data[key]

dict_keys(['x_vec', 'n_groups', 'optimal_order_paulis', 'optimal_order_qubits', 'optimal_graph'])


In [3]:
n_x = len(x_vec)

n_std = np.zeros(n_x)
n_avg = np.zeros(n_x)
n_min = np.zeros(n_x)
n_max = np.zeros(n_x)

for i in range(n_x):
    n_std[i] = np.std(n_groups[i])
    n_avg[i] = np.mean(n_groups[i])
    n_min[i] = np.min(n_groups[i])
    n_max[i] = np.max(n_groups[i])

In [12]:
fig, ax = plt.subplots()
ax.plot(x_vec, n_avg)
ax.fill_between(x_vec, n_avg - n_std, n_avg + n_std, alpha=0.25)
ax.plot(x_vec, n_min, '--')
ax.plot(x_vec, n_max, '--')
ax.set_xlabel('x')
ax.set_ylabel('# of groups')
ax.set_xlim([x_vec[0], x_vec[-1]]);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
index_min = np.argmin(n_min)
print('The minumum number of groups is {:.0f}, obtained for x = {:.3f}'.format(n_min[index_min], x_vec[index_min]))

The minumum number of groups is 36, obtained for x = 0.842


In [11]:
plt.figure()
nx.draw(optimal_graph[index_min])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …